## SalishSeaCast Nowcast Timeseries Analysis
This notebook loads hourly Nowcast results 1 day at a time, and extracts selected slices into numpy arrays. The notebook itself is a development environment for the script `analysis-ben/scripts/prepare_nowcast_timeseries.py`.

In [88]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import progressbar
import pickle
from scipy.io import savemat, loadmat
from salishsea_tools import tidetools, viz_tools, nc_tools, geo_tools

%matplotlib inline

### Load NEMO results
Timerange

In [2]:
# Timerange
timerange = ['2017 Jan 1 00:00', '2017 Jan 3 23:00']
timeslice = slice(timerange[0], timerange[1])

Build NEMO mask and grid arrays

In [3]:
def reshape_grid(tmask, deptht, gridY, gridX, index=0, dim=0, spacing=1):
    """Prepare the mask and grid for the selected timeseries slice, and reshape into 1 spatial dimension
    """
    
    tmask = tmask.take(index, axis=dim)[::spacing, ::spacing]
    ngrid = tmask.shape[0] * tmask.shape[1]
    tmask = tmask.reshape(ngrid)
    ngrid_water = tmask.sum()
    gridX = gridX.take(index, axis=dim)[::spacing, ::spacing].reshape(ngrid)[tmask]
    gridY = gridY.take(index, axis=dim)[::spacing, ::spacing].reshape(ngrid)[tmask]
    deptht = deptht.take(index, axis=dim)[::spacing, ::spacing].reshape(ngrid)[tmask]
    
    GRIDMASK = {'tmask': tmask, 'deptht': deptht, 'gridY': gridY,
                'gridX': gridX, 'ngrid': ngrid, 'ngrid_water': ngrid_water}
    
    return GRIDMASK

In [13]:
def reshape_data(data, GRIDMASK, dim, index=0, spacing=1, unstagger_dim=None):
    """
    """
    
    field = data.isel(**{dim: index})
    if unstagger_dim is not None:
        field = viz_tools.unstagger_xarray(field, unstagger_dim)
    field = field.values[:, ::spacing, ::spacing]
    field = field.reshape((-1, GRIDMASK['ngrid']))
    field_trim = np.zeros((field.shape[0], GRIDMASK['ngrid_water']))
    
    return field, field_trim

In [5]:
def calc_rho(Sal, TempC, P):
    """ Calculate rho: Based on SOG code
    """
    
    # Calculate the square root of the salinities
    sqrSal = np.sqrt(Sal)

    # Calculate the density profile at the grid point depths
    # Pure water density at atmospheric pressure
    # (Bigg P.H., (1967) Br. J. Applied Physics 8 pp 521-537)
    R1 = ((((6.536332e-9 * TempC - 1.120083e-6) * TempC + 1.001685e-4)
           * TempC - 9.095290e-3) * TempC + 6.793952e-2) * TempC - 28.263737
    R2 = (((5.3875e-9 * TempC - 8.2467e-7) * TempC + 7.6438e-5)
          * TempC - 4.0899e-3) * TempC + 8.24493e-1
    R3 = (-1.6546e-6 * TempC + 1.0227e-4) * TempC - 5.72466e-3

    # International one-atmosphere equation of state of seawater
    SIG = (4.8314e-4 * Sal + R3 * sqrSal + R2) * Sal + R1

    # Specific volume at atmospheric pressure
    V350P = 1.0 / 1028.1063
    SVA   = -SIG * V350P / (1028.1063 + SIG)

    # Density anomoly at atmospheric pressure
    rho = 28.106331 - SVA / (V350P * (V350P + SVA)) + 1000
    
    return rho

In [7]:
# Get mask and grid files
mask_NEMO = xr.open_dataset('/ocean/bmoorema/research/MEOPAR/NEMO-forcing/grid/mesh_mask_downbyone2.nc')

GRIDMASK = {}

# Mask
tmask = mask_NEMO.tmask.isel(t=0).values.astype(bool)
tmask[:, 750:, :] = 0
tmask[:, :350, :] = 0
tmask[:, :, :100] = 0

# Grid and depth
deptht, gridY, gridX = np.meshgrid(mask_NEMO.gdept_1d.isel(t=0), mask_NEMO.y, mask_NEMO.x, indexing='ij')
deptht = mask_NEMO.gdept_0.isel(t=0).values

# Nowcast domain slice parameters
indices = [0, 0, 20, 450, 520, 680]
spacing = [1, 5, 5, 1, 1, 1]
dims = [0, 0, 0, 1, 1, 1]

# Create mask and grid arrays for Nowcast domain slices
for index, space, dim in zip(indices, spacing, dims):
    
    # Depth 20 is index 18
    if index == 20:
        ii = 18
    else:
        ii = index
        
    # Store mask and grid arrays in dict
    GRIDMASK[f'spc{space}_{index}'] = reshape_grid(tmask, deptht, gridY, gridX, index=ii, dim=dim, spacing=space)

# dz
deptht = deptht.reshape(-1, GRIDMASK['spc1_0']['ngrid'])[:, GRIDMASK['spc1_0']['tmask']]
dz = np.diff(deptht, axis=0)

Build GEM mask

In [4]:
# Open NetCDF Files
grid_NEMO = xr.open_dataset('/ocean/bmoorema/research/MEOPAR/NEMO-forcing/grid/bathy_downonegrid2.nc')
grid_GEM  = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaAtmosphereGridV1')

# Preallocate
ngrid_GEM = grid_GEM.gridX.shape[0] * grid_GEM.gridY.shape[0]
mask_GEM = np.zeros(ngrid_GEM, dtype=int)

# Evaluate each point on GEM grid
for index, coords in enumerate(zip(
    grid_GEM.longitude.values.reshape(ngrid_GEM) - 360,
    grid_GEM.latitude.values.reshape(ngrid_GEM))):
    
    j, i = geo_tools.find_closest_model_point(coords[0], coords[1], grid_NEMO.nav_lon, grid_NEMO.nav_lat)
    if j is np.nan or i is np.nan:
        mask_GEM[index] = 0
    else:
        mask_GEM[index] = mask.tmask.isel(z=0, x=i, y=j).values

# Reshape
mask_GEM = mask_GEM.reshape(grid_GEM.longitude.shape)

# Mask out PS and JdF
mask_GEM[200:, :] = 0
mask_GEM[:110, :] = 0
mask_GEM[:140, :125] = 0

# Reshape
mask_GEM = mask_GEM.reshape(ngrid_GEM)

# Number of water points
ngrid_GEM_water = mask_GEM.sum()

# Convert to bool
mask_GEM = mask_GEM.astype(bool)

# Pack into GRIDMASK
GRIDMASK['GEM'] = {'tmask': mask_GEM, 'ngrid': ngrid_GEM, 'ngrid_water': ngrid_GEM_water}

Get GEM data

In [5]:
# Open HRDPS wind forcing from ERDDAP
wind = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')

# Reshape velocity fields
u_wind = wind.sel(time=timeslice).u_wind.values.reshape(-1, GRIDMASK['GEM']['ngrid'])
v_wind = wind.sel(time=timeslice).v_wind.values.reshape(-1, GRIDMASK['GEM']['ngrid'])

# Preallocated trimmed fields
u_wind_trim = np.zeros((u_wind.shape[0], GRIDMASK['GEM']['ngrid_water']))
v_wind_trim = np.zeros((v_wind.shape[0], GRIDMASK['GEM']['ngrid_water']))

# Trim fields
for tindex, timerows in enumerate(zip(u_wind, v_wind)):
    u_wind_trim[tindex, :] = timerows[0][GRIDMASK['GEM']['tmask']]
    v_wind_trim[tindex, :] = timerows[1][GRIDMASK['GEM']['tmask']]
    
# Pack into dictionary
GEM = {
    'u_wind': u_wind_trim,
    'v_wind': v_wind_trim
}

Build NEMO filenames

In [8]:
# Build filename lists
filenames_T = nc_tools.make_filename_list(timerange, 'T', model='nowcast-green', resolution='h')
filenames_U = nc_tools.make_filename_list(timerange, 'U', model='nowcast-green', resolution='h')
filenames_V = nc_tools.make_filename_list(timerange, 'V', model='nowcast-green', resolution='h')

# Predefine storage dictionaries
NEMO, FULL, TRIM = {}, {}, {}

Main loop

In [47]:
# Main loop
with progressbar.ProgressBar(max_value=len(filenames_T)) as bar:
    for i, filename in bar(enumerate(zip(filenames_U, filenames_V, filenames_T))):
        
        # Open NEMO results into numpy arrays
        U = xr.open_dataset(filename[0]).vozocrtx
        V = xr.open_dataset(filename[1]).vomecrty
        T = xr.open_dataset(filename[2]).votemper
        S = xr.open_dataset(filename[2]).vosaline
        
        # Calculate density
        rho = calc_rho(S, T, mask_NEMO.gdept_0.isel(t=0)).values
        rho = rho.reshape((-1, rho.shape[1], GRIDMASK['spc1_0']['ngrid']))
        TRIM['pycnocline'] = np.zeros((S.shape[0], GRIDMASK['spc1_0']['ngrid_water']))
        
        # Reshape Parameters
        indices = [0, 20, 450, 520, 680]
        spacing = [5, 5, 1, 1, 1]
        dims = ['depth', 'depth', 'y', 'y', 'y']
        tracers = [False, False, True, True, True]
        
        # Slice and reshape NEMO results and allocate trimmed output arrays
        for index, space, dim, tracer in zip(indices, spacing, dims, tracers):
            
            # Hash slice dims
            if dim is 'depth':
                dimu, dimv, dimt = 'depthu', 'depthv', 'deptht'
            else:
                dimu, dimv, dimt = dim, dim, dim
            
            # Reshape tracers
            if tracer:
                FULL[f'T{index}'], TRIM[f'T{index}'] = reshape_data(
                    T, GRIDMASK[f'spc{space}_{index}'], dimt, index=index, spacing=space)
                FULL[f'S{index}'], TRIM[f'S{index}'] = reshape_data(
                    S, GRIDMASK[f'spc{space}_{index}'], dimt, index=index, spacing=space)
            
            # Reshape velocity
            if space > 1:
                dimx, dimy = 'x', 'y'
            else:
                dimx, dimy = None, None
            FULL[f'U{index}'], TRIM[f'U{index}'] = reshape_data(
                U, GRIDMASK[f'spc{space}_{index}'], dimu, index=index, spacing=space, unstagger_dim=dimx)
            FULL[f'V{index}'], TRIM[f'V{index}'] = reshape_data(
                V, GRIDMASK[f'spc{space}_{index}'], dimv, index=index, spacing=space, unstagger_dim=dimy)
        
        # Trim Land Points and calculate halocline depth
        for tindex, timestamp in enumerate(U.time_counter):
            
            # Loop through slice parameters
            for index, space, tracer in zip(indices, spacing, tracers):
                
                # Trim tracers
                if tracer:
                    TRIM[f'T{index}'][tindex, :] = FULL[f'T{index}'][tindex, :][GRIDMASK[f'spc{space}_{index}']['tmask']]
                    TRIM[f'S{index}'][tindex, :] = FULL[f'S{index}'][tindex, :][GRIDMASK[f'spc{space}_{index}']['tmask']]
                
                # Trim velocity
                TRIM[f'U{index}'][tindex, :] = FULL[f'U{index}'][tindex, :][GRIDMASK[f'spc{space}_{index}']['tmask']]
                TRIM[f'V{index}'][tindex, :] = FULL[f'V{index}'][tindex, :][GRIDMASK[f'spc{space}_{index}']['tmask']]
            
            # Pycnocline depth
            idrhodz = (np.diff(rho[tindex][:, GRIDMASK['spc1_0']['tmask']], axis=0) / dz).argmax(axis=0)
            TRIM['pycnocline'][tindex, :] = np.array([depth[dindex] for dindex, depth in zip(idrhodz, deptht.T)])
        
        # Concatenate into storage arrays
        for index, tracer in zip(indices, tracers):
            
            # Assign if first file
            if i == 0:
                if tracer:
                    NEMO[f'T{index}'] = TRIM[f'T{index}']
                    NEMO[f'S{index}'] = TRIM[f'S{index}']
                
                NEMO[f'U{index}'] = TRIM[f'U{index}']
                NEMO[f'V{index}'] = TRIM[f'V{index}']
            
            # Otherwise concatenate
            else:
                if tracer:
                    NEMO[f'T{index}'] = np.concatenate([NEMO[f'T{index}'], TRIM[f'T{index}']], axis=0)
                    NEMO[f'S{index}'] = np.concatenate([NEMO[f'S{index}'], TRIM[f'S{index}']], axis=0)
                
                NEMO[f'U{index}'] = np.concatenate([NEMO[f'U{index}'], TRIM[f'U{index}']], axis=0)
                NEMO[f'V{index}'] = np.concatenate([NEMO[f'V{index}'], TRIM[f'V{index}']], axis=0)
        
        # Concatenate pycnocline depth
        NEMO['pycnocline'] = TRIM['pycnocline']
        NEMO['pycnocline'] = np.concatenate([NEMO['pycnocline'], TRIM['pycnocline']], axis=0)
        
        # Update progress bar
        bar.update(i)

100% (3 of 3) |###########################| Elapsed Time: 0:07:49 Time: 0:07:49


KeyboardInterrupt: 

In [96]:
savemat('/ocean/bmoorema/research/MEOPAR/analysis-ben/data/NEMO_2016.mat', NEMO)

In [97]:
fid = open('/ocean/bmoorema/research/MEOPAR/analysis-ben/data/GRIDMASK_2016', 'wb') 
pickle.dump(GRIDMASK, fid)   
fid.close()